In [1]:
import re
import pandas as pd
import requests
import openpyxl
from time import sleep
from random import uniform
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [13]:
def get_page(url):
    try:
        response = requests.get(url, headers=headers, timeout=35)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'lxml')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

In [4]:
def scrape_criminals(url):
    soup = get_page(url)
    hrefs = soup.find_all("a", class_="branches-list__item iconf-share2")
    links = [a['href'] for a in hrefs if a.has_attr('href')]
    soup_links = []
    for link in links:
        soup_links.append(get_page(link))
        sleep(uniform(1, 2))
    links_next = []
    for i in tqdm(range(len(soup_links))):
        menu_links = []
        menu_links = soup_links[i].find_all("a", class_="b-menu-link")
        for link in menu_links:
            if link.text == 'Внимание, розыск!' and link.has_attr('href'):
                links_next.append(f"{links[i]}{link['href']}")
        sleep(uniform(1, 2))
    soup_links_next = []
    for link in links_next:
        soup_links_next.append(get_page(link))
        sleep(uniform(1, 2))
    links_next_next = []
    for i in tqdm(range(len(soup_links_next))):
        main_links = []
        main_links = soup_links_next[i].find_all("li")
        for link in main_links:
            a_tag = link.find('a')
            if link.text == 'Подозреваемые в совершении преступлений' and a_tag.has_attr('href'):
                links_next_next.append(f"{links[i]}{a_tag['href']}")
        sleep(uniform(1, 2))
    links_next_next_full = []
    for i in tqdm(range(len(links_next_next))):
        j = 1
        while True:
            link = f"{links_next_next[i]}/{j}/"
            try:
                response = requests.get(link, headers=headers)
                if response.status_code == 200:
                    j += 1
                    links_next_next_full.append(link)
                    print(link)
                else:
                    break
            except requests.exceptions.RequestException as e:
                print(f"Error fetching {link}: {e}")
                break
        sleep(uniform(1, 2))
    soup_links_next_next_full = []
    for link in links_next_next_full:
        soup_links_next_next_full.append(get_page(link))
        sleep(uniform(1, 2))
    criminals = []
    for i in range(len(soup_links_next_next_full)):
        criminals.append(soup_links_next_next_full[i].find_all("div", class_="bl-item clearfix"))
    return criminals, links_next_next_full

In [17]:
def scrape_criminal_data(criminals, links_next_next_full):
    pattern = r"^(?:[^/]*\/){3}(.*)"
    pattern_full_text = r'^\n$'
    pattern_base_url = r'^(https?://[^/]+)'
    pattern_drop_end = r'^(.*)\/.*?\/$'
    df = pd.DataFrame(columns=['Title', 'Text', 'Image'])
    for i in tqdm(range(len(criminals))):
        for j in range(len(criminals[i])):
            title = ""
            image_url = ""
            full_text = ""
            title_div = criminals[i][j].find("div", class_="bl-item-title")
            if title_div:
                a_tag = title_div.find("a")
                if a_tag:
                    href_criminal = a_tag
                    if href_criminal:
                        match = re.search(pattern, href_criminal['href'])
                        if match:
                            extracted_part = match.group(1)
                            links_next_next_full_drop_end_group = re.search(pattern_drop_end, links_next_next_full[i])
                            links_next_next_full_drop_end = links_next_next_full_drop_end_group.group(1)
                            full_path_to_criminal = f'{links_next_next_full_drop_end}/{extracted_part}'
                            criminal_html = get_page(full_path_to_criminal)
                            if criminal_html:
                                criminal_info_blocks = criminal_html.find("section", class_="b-container-center")
                                title_element = criminal_info_blocks.find("h1", class_="b-topic t-h1 m_b4")
                                if title_element:
                                    title = title_element.text
                                else:
                                    criminal_info_blocks = criminal_html.find("div", id="news_position")
                                    title_element = criminal_info_blocks.find("div", class_="header")
                                    if title_element:
                                        title_find = title_element.find("h1")
                                        if title_find:
                                            title = title_find.text
                                article = criminal_info_blocks.find("article", class_="c-detail m_b4")
                                if article:
                                    match = re.match(pattern_base_url, links_next_next_full[i])
                                    if match:
                                        base_url = match.group(1)
                                    img_tag = article.find("img")
                                    if img_tag:
                                        image_url = f'{base_url}{img_tag.get("src")}'
                                    paragraphs = article.find_all(['p', 'div', 'span', 'br'])
                                    full_text = "\n".join([p.text.strip() for p in paragraphs])
                                    if re.match(pattern_full_text, full_text):
                                        full_text = article.text
                                else:
                                    div = criminal_info_blocks.find("div", class_="c-detail text")
                                    if div:
                                        match = re.match(pattern_base_url, links_next_next_full[i])
                                        if match:
                                            base_url = match.group(1)
                                        img_tag = div.find("img")
                                        if img_tag:
                                            image_url = f'{base_url}{img_tag.get("src")}'
                                        paragraphs = div.find_all(['p', 'div', 'span', 'br'])
                                        full_text = "\n".join([p.text.strip() for p in paragraphs])
                                df.loc[len(df)] = title, full_text, image_url
            sleep(uniform(1, 2))
    return df

In [6]:
url = "https://sledcom.ru/sk_russia/structure/mapregions/"

In [7]:
criminals, links_next_next_full = scrape_criminals(url)

  0%|          | 0/76 [00:00<?, ?it/s]

http://belgorod.sledcom.ru/attention/suspected_persons/1/


  1%|▏         | 1/76 [00:03<04:44,  3.79s/it]

http://vladimir.sledcom.ru/attention/suspected_persons/1/


  3%|▎         | 2/76 [00:06<04:05,  3.32s/it]

...

 97%|█████████▋| 74/76 [05:17<00:06,  3.29s/it]

http://kherson.sledcom.ru/attention/suspected_persons/1/


 99%|█████████▊| 75/76 [05:20<00:03,  3.30s/it]

http://zaporozhye.sledcom.ru/attention/suspected_persons/1/


100%|██████████| 76/76 [05:23<00:00,  4.26s/it]


In [18]:
df = scrape_criminal_data(criminals, links_next_next_full)

100%|██████████| 185/185 [1:00:45<00:00, 19.71s/it]


In [19]:
df

,Title,Text,Image
0,Внимание! Розыск! Устанавливается подозреваемы...,Приметы напавшего по состоянию на 2015 год: на...,http://vladimir.sledcom.ru/upload/site16/docum...
1,"Пономарев Игорь Сергеевич, 27.07.1975 г.р.","\nуголовное дело № 1628, возбуждено 28.07.2005...",
2,"Петросян Арарат Матевосович, 2.04.1962 г.р.","\n1. уголовное дело № 1636, возбуждено 14.08.2...",http://vladimir.sledcom.ru/upload/site16/6yzCh...
3,Мусаев Асиф Джалал оглы,"\n1. уголовное дело № 792, возбуждено 1.05.20...",http://vladimir.sledcom.ru/upload/site16/OAoVX...
4,Тухтаров Баходур Хангеевич,"\nуголовное дело № 2479, возбуждено 10.08.2005...",
...,...,...,...
1491,ВНИМАНИЕ РОЗЫСК,\n\n\nВтором отделом по расследованию особо ва...,http://zaporozhye.sledcom.ru/upload/site105/do...
1492,ВНИМАНИЕ РОЗЫСК,\n\n\nПервым отделом по расследованию особо ва...,http://zaporozhye.sledcom.ru/upload/site105/do...
1493,ВНИМАНИЕ РОЗЫСК,\n\n\nУроженка г. Бердянск Запорожской области...,http://zaporozhye.sledcom.ru/upload/site105/do...
1494,ВНИМАНИЕ РОЗЫСК,\n\n\nПервым отделом по расследованию особо ва...,http://zaporozhye.sledcom.ru/upload/site105/do...


In [20]:
df.to_excel('sledcom.xlsx')

In [21]:
df.to_csv('sledcom.csv')